- pandas: Used for data manipulation and analysis.

- numpy: Used for numerical operations.

In [1]:
import pandas as pd
import numpy as np

- Load the dataset from an Excel file named 'MOOD TRAK.xlsx' into a DataFrame.

- Display the first few rows of the DataFrame to inspect the data.

In [2]:
df=pd.read_excel('MOOD TRAK.xlsx')
df.head()

,Time Stamp,Day,Activities,Start,End,Quality,Note
0,2024-05-01 07:00:00,Wednesday,sleep,23:00:00,05:00:00,okay,Bangun pagi untuk sholat subuh.
1,2024-05-01 07:00:00,Wednesday,selfcare,05:00:00,05:30:00,good,"Mandi dan bersiap, air dingin di pagi hari tid..."
2,2024-05-01 07:00:00,Wednesday,eating,05:30:00,06:00:00,good,Sarapan pagi dengan roti bakar dan susu.
3,2024-05-01 07:00:00,Wednesday,workout,06:00:00,07:00:00,excellent,Jogging sekitar kos-kosan. Bertemu banyak anak...
4,2024-05-01 07:00:00,Wednesday,traveling,07:00:00,07:30:00,excellent,"Berangkat ke kampus, perjalanan sekitar 2 km d..."


- Drop the 'Note' column as it is not relevant to the model.

- Convert the 'Time Stamp' column to datetime format and keep only the date part.

- Convert 'Start' and 'End' columns to datetime format and keep only the time part.

- Calculate the duration of each activity as the difference between 'End' and 'Start' times.

- Adjust for activities that span past midnight by adding a day to negative durations.

- Convert the duration from timedelta to minutes.



In [10]:
# Drop 'Note' columns because its not used in this model
df_new=df.drop(columns=['Note'])

# Convert time columns to datetime
df_new['Time Stamp'] = pd.to_datetime(df_new['Time Stamp']).dt.date

# Convert 'Start' and 'End' to datetime
df_new['Start'] = pd.to_datetime(df_new['Start'], format='%H:%M:%S').dt.time
df_new['End'] = pd.to_datetime(df_new['End'], format='%H:%M:%S').dt.time

# Calculate duration
df_new['Duration'] = pd.to_timedelta(df_new['End'].astype(str)) - pd.to_timedelta(df_new['Start'].astype(str))
df_new['Duration'] = df_new['Duration'].apply(lambda x: x + pd.Timedelta(days=1) if x.days < 0 else x)
df_new['Duration'] = df_new['Duration'].apply(lambda x: int(pd.to_timedelta(x).total_seconds() // 60)) #Hour to Minutes

df_new

,Time Stamp,Day,Activities,Start,End,Quality,Duration
0,2024-05-01,Wednesday,sleep,23:00:00,05:00:00,okay,360
1,2024-05-01,Wednesday,selfcare,05:00:00,05:30:00,good,30
2,2024-05-01,Wednesday,eating,05:30:00,06:00:00,good,30
3,2024-05-01,Wednesday,workout,06:00:00,07:00:00,excellent,60
4,2024-05-01,Wednesday,traveling,07:00:00,07:30:00,excellent,30
...,...,...,...,...,...,...,...
1215,2024-08-06,Tuesday,entertainment,17:45:00,19:00:00,okay,75
1216,2024-08-06,Tuesday,eating,19:00:00,19:30:00,okay,30
1217,2024-08-06,Tuesday,study,19:30:00,21:00:00,okay,90
1218,2024-08-06,Tuesday,dating,21:00:00,23:00:00,okay,120


Group the data by 'Time Stamp' and 'Activities' and calculate the sum of durations for each combination.

Pivot the aggregated data to have 'Time Stamp' as the index and activities as columns.

Replace missing values with 0 to handle days when certain activities were not performed.



In [11]:
# Aggregate daily data
daily_summary = df_new.groupby(['Time Stamp', 'Activities']).agg({'Duration': 'sum'}).reset_index()

daily_summary_pivot = daily_summary.pivot(index='Time Stamp', columns='Activities', values='Duration').fillna(0)
daily_summary_pivot.reset_index(inplace=True)
daily_summary_pivot

Activities,Time Stamp,dating,eating,entertainment,selfcare,sleep,study,traveling,work,workout
0,2024-05-01,0.0,105.0,165.0,45.0,360.0,630.0,135.0,0.0,60.0
1,2024-05-02,260.0,135.0,0.0,45.0,300.0,480.0,145.0,0.0,75.0
2,2024-05-03,0.0,120.0,180.0,60.0,480.0,480.0,60.0,0.0,60.0
3,2024-05-04,210.0,90.0,300.0,45.0,795.0,0.0,0.0,0.0,0.0
4,2024-05-05,150.0,85.0,465.0,0.0,300.0,240.0,199.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,120.0,120.0,75.0,30.0,450.0,90.0,75.0,465.0,45.0
94,2024-08-03,120.0,120.0,75.0,30.0,450.0,90.0,75.0,435.0,45.0
95,2024-08-04,120.0,120.0,75.0,30.0,420.0,90.0,75.0,435.0,45.0
96,2024-08-05,120.0,120.0,75.0,30.0,420.0,90.0,75.0,465.0,45.0


In [9]:
# Define normal ranges for activities
normal_ranges = {
    'dating': (30, 60),
    'eating': (90, 120),
    'entertainment': (60, 180),
    'selfcare': (30, 60),
    'sleep': (420, 540),
    'study': (120, 240),
    'traveling': (60, 120),
    'work': (420, 540),
    'workout': (30, 60)
}

def generate_recommendations(df, normal_ranges):
    recommendations = []

    for index, row in df.iterrows():
        daily_recommendations = {'Time Stamp': row['Time Stamp']}
        for activity, (min_time, max_time) in normal_ranges.items():
            actual_time = row[activity]
            if actual_time < min_time:
                daily_recommendations[activity] = f"Increase on your {activity} by {min_time - actual_time} minutes"
            elif actual_time > max_time:
                daily_recommendations[activity] = f"Decrease on your {activity} by {actual_time - max_time} minutes"
            else:
                daily_recommendations[activity] = "Optimal"
        recommendations.append(daily_recommendations)

    return pd.DataFrame(recommendations)

# Generate recommendations
recommendations_df = generate_recommendations(daily_summary_pivot, normal_ranges)
recommendations_df

,Time Stamp,dating,eating,entertainment,selfcare,sleep,study,traveling,work,workout
0,2024-05-01,Increase on your dating by 30.0 minutes,Optimal,Optimal,Optimal,Increase on your sleep by 60.0 minutes,Decrease on your study by 390.0 minutes,Decrease on your traveling by 15.0 minutes,Increase on your work by 420.0 minutes,Optimal
1,2024-05-02,Decrease on your dating by 200.0 minutes,Decrease on your eating by 15.0 minutes,Increase on your entertainment by 60.0 minutes,Optimal,Increase on your sleep by 120.0 minutes,Decrease on your study by 240.0 minutes,Decrease on your traveling by 25.0 minutes,Increase on your work by 420.0 minutes,Decrease on your workout by 15.0 minutes
2,2024-05-03,Increase on your dating by 30.0 minutes,Optimal,Optimal,Optimal,Optimal,Decrease on your study by 240.0 minutes,Optimal,Increase on your work by 420.0 minutes,Optimal
3,2024-05-04,Decrease on your dating by 150.0 minutes,Optimal,Decrease on your entertainment by 120.0 minutes,Optimal,Decrease on your sleep by 255.0 minutes,Increase on your study by 120.0 minutes,Increase on your traveling by 60.0 minutes,Increase on your work by 420.0 minutes,Increase on your workout by 30.0 minutes
4,2024-05-05,Decrease on your dating by 90.0 minutes,Increase on your eating by 5.0 minutes,Decrease on your entertainment by 285.0 minutes,Increase on your selfcare by 30.0 minutes,Increase on your sleep by 120.0 minutes,Optimal,Decrease on your traveling by 79.0 minutes,Increase on your work by 420.0 minutes,Increase on your workout by 30.0 minutes
...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,Decrease on your dating by 60.0 minutes,Optimal,Optimal,Optimal,Optimal,Increase on your study by 30.0 minutes,Optimal,Optimal,Optimal
94,2024-08-03,Decrease on your dating by 60.0 minutes,Optimal,Optimal,Optimal,Optimal,Increase on your study by 30.0 minutes,Optimal,Optimal,Optimal
95,2024-08-04,Decrease on your dating by 60.0 minutes,Optimal,Optimal,Optimal,Optimal,Increase on your study by 30.0 minutes,Optimal,Optimal,Optimal
96,2024-08-05,Decrease on your dating by 60.0 minutes,Optimal,Optimal,Optimal,Optimal,Increase on your study by 30.0 minutes,Optimal,Optimal,Optimal
